In [ ]:
! pip install -r requirements.txt --quiet

# Semantic Kernel Sequential Orchestration

In sequential orchestration, agents are organized in a pipeline. Each agent processes the task in turn, passing its output to the next agent in the sequence. 

🔗 [Sequential Orchestration](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/agent-orchestration/sequential?pivots=programming-language-python)

In [1]:
from semantic_kernel.agents import SequentialOrchestration
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent,ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from dotenv import load_dotenv
from os import environ
import asyncio
from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin

load_dotenv(override=True)

kernel = Kernel()


In [2]:
chatCompletion = AzureChatCompletion(
    service_id="chat",
    deployment_name=environ["AZURE_OPENAI_MODEL"],
    base_url=environ["AZURE_OPENAI_ENDPOINT"],
    api_key=environ["AZURE_OPENAI_API_KEY"] )


reasoningCompletion = AzureChatCompletion(
    service_id="reasoning",
    deployment_name='o3-mini',
    base_url=environ["AZURE_OPENAI_ENDPOINT"],
    api_key=environ["AZURE_OPENAI_API_KEY"] )



In [3]:
sales_plugin = MCPStreamableHttpPlugin(
    name="sales",
    url=f"{environ['MCP_SERVER_URL']}",
)

await sales_plugin.connect()


In [10]:

# Sales Data Agent
sales_data_agent = ChatCompletionAgent(
    name="SalesDataAgent",
    instructions=(
        """You are a Sales Data Agent. Your job is to:
        - Dynamically choose and call the appropriate tool(s) to answer user questions about sales, customers, or products.
        - Always include customer names alongside IDs in any output.
        - Always display monetary amounts in $USD (e.g., $123.45).
        - Use filters and aggregations as needed to generate insights from sales orders, products, and customers.
        - Compute totals, revenue, discounts, and other metrics from nested order data.
        - Return **complete, detailed outputs** without summarization, so that all data is available for downstream analysis.
        - Preserve all intermediate results and raw values in your response.
        - Avoid hardcoding analytics; rely on the MCP tools and their parameters.
        - Clarify ambiguous queries before performing analysis.
        - Do not ask the user follow-up questions; only return the requested records.
        - Treat the tools as the source of truth; do not expose raw database internals.
        - return output in table format when possible for consumption by the next agent.
        """
    ),
    plugins=[sales_plugin],
    service=chatCompletion,
)

# Data Analysis Agent
data_analysis_agent = ChatCompletionAgent(
    name="DataAnalysisAgent",
    instructions=(
        """You are a Data Analysis Agent. Given the sales data, your job is to:
        - Analyze and interpret the sales data provided by the SalesDataAgent.
        - Identify trends, patterns, and insights from the sales data.
        - Provide actionable recommendations based on the analysis.
        - Summarize key findings in a clear and concise manner.
        - Ensure that all analyses are relevant to the user's query and objectives.
        - Provide Key Findings, Insights, Recommendations and a Summary in your final output.
        """
    ),
      service=reasoningCompletion,
)

### Optional: Observe Agent Responses

In [11]:
from semantic_kernel.contents import ChatMessageContent

def agent_response_callback(message: ChatMessageContent) -> None:
    print(f"# {message.name}\n{message.content}")

### Set Up the Sequential Orchestration

In [12]:
agents = [sales_data_agent, data_analysis_agent]
sequential_orchestration = SequentialOrchestration(
    members=agents,
    agent_response_callback=agent_response_callback,
)

### Start the Runtime

In [13]:
from semantic_kernel.agents.runtime import InProcessRuntime

runtime = InProcessRuntime()
runtime.start()

# SalesDataAgent

# SalesDataAgent

# SalesDataAgent
Based on the sales data:

### Total Sales per Customer

| Customer ID | Customer Name | Total Sales ($USD) |
|-------------|---------------|--------------------|
| 1           | GM            | 4,104.65          |
| 4           | NAPA          | 3,952.93          |
| 3           | Bosch         | 2,939.52          |
| 2           | AutoZone      | 3,168.38          |
| 0           | Ford          | 1,096.42          |

### Customer with the Highest Sales: GM (Customer ID: 1)
- **Total Sales ($USD):** 4,104.65

---

### Top 3 Products Purchased by GM

| Product Name | Quantity Purchased | Total Revenue ($USD) |
|--------------|--------------------|----------------------|
| Brake Pad    | 143                | 7,143.00            |
| Spark Plug   | 94                 | 767.04              |
| Oil Filter   | 65                 | 975.00              |

Let me know if you'd like further analysis or details!
# DataAnalysisAgent
### Key Find

In [14]:
orchestration_result = await sequential_orchestration.invoke(
    task="Which customer has the highest total sales, and what are the top 3 products they purchased?",
    runtime=runtime,
)

In [15]:
value = await orchestration_result.get(timeout=20)
print(f"***** Final Result *****\n{value}")

***** Final Result *****
### Key Findings:
1. **Top Performing Customer:**
   - GM is the highest spender, with total sales of **$4,104.65**, followed by NAPA (**$3,952.93**) and AutoZone (**$3,168.38**).
   - Ford has the lowest sales contribution at **$1,096.42**.

2. **Product Insights:**
   - Brake Pads are GM's most purchased product (143 units, contributing **$7,143.00** in revenue).
   - Spark Plugs and Oil Filters followed, generating **$767.04** and **$975.00**, respectively.

3. **Revenue Concentration:**
   - GM and NAPA collectively contribute **40%** of the overall revenue, highlighting a reliance on a few high-spending customers.

4. **Low Representation:**
   - Ford represents significant underperformance, with a total sales contribution far below its counterparts.

---

### Insights:
1. GM is likely purchasing a high volume of essential consumables (brake pads, spark plugs, oil filters), suggesting these items drive repeat purchases.
2. NAPA and AutoZone also show stron

In [16]:
await runtime.stop_when_idle()